Now it is time to complicate things and use H2O GBM and RF with grid search, and of course XGBoost.

Thi isi a great article for tunning XGboost: http://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

# H2O - GLM, GBM, NN, RF

In [1]:
import pandas as pd
import numpy as np
import time
import csv
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 8
import math

_start_time = time.time()

def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

In [2]:
import h2o
import time
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator


from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [3]:
#now we load our modified train and test set

tic()
#train = pd.read_csv('./input-data/train_modified.csv', nrows = 500000)
#test = pd.read_csv('./input-data/test_modified.csv', nrows = 500000)
train = pd.read_csv('./input-data/train_modified.csv',
                    dtype = {'Semana': 'int8',
                            'Agencia_ID': 'uint16',
                            'Canal_ID': 'int8',
                            'Ruta_SAK': 'int32',
                            'Cliente_ID': 'int32',
                            'Producto_ID': 'int32',
                            'log_target':  'float64',
                            'Log_Target_mean_lag1': 'float64',
                            'Log_Target_mean_lag2': 'float64',
                            'Log_Target_mean_lag3': 'float64',
                            'Log_Target_mean_lag4': 'float64',
                            'Lags_sum': 'float64',
                            'pairs_mean':  'float64',
                            'brand': 'uint16',
                            'cluster': 'uint16',
                            'Qty_Ruta_SAK_Bin': 'int8',
                            'ZipCode': 'uint16'},
                   )

                    
test = pd.read_csv('./input-data/test_modified.csv',
                    dtype = {'id': 'uint32',
                            'Semana': 'int8',
                            'Agencia_ID': 'uint16',
                            'Canal_ID': 'int8',
                            'Ruta_SAK': 'int32',
                            'Cliente_ID': 'int32',
                            'Producto_ID': 'int32',
                            'Log_Target_mean_lag1': 'float64',
                            'Log_Target_mean_lag2': 'float64',
                            'Log_Target_mean_lag3': 'float64',
                            'Log_Target_mean_lag4': 'float64',
                            'Lags_sum': 'float64',
                            'pairs_mean': 'float64',
                            'brand': 'uint16',
                            'cluster': 'uint16',
                            'Qty_Ruta_SAK_Bin': 'int8',
                            'ZipCode': 'uint16'},
                      )
tac()

Time passed: 0.0hour:1.0min:34.0sec


In [4]:
# Connect to a cluster
h2o.init()

Connecting to H2O server at http://localhost:54321... successful!


H2O cluster uptime:,4 hours 53 mins
H2O cluster version:,3.10.0.1
H2O cluster version age:,12 days
H2O cluster name:,LOCAL SERVICE
H2O cluster total nodes:,1
H2O cluster free memory:,10.30 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster is healthy:,True
H2O cluster is locked:,False
H2O connection url:,http://localhost:54321


In [ ]:
train_hex = h2o.H2OFrame(train)
test_hex = h2o.H2OFrame(test)

In [ ]:
train_hex.describe()

In [ ]:
# ----------
# Fit a model on train; using test and validation

# Function for doing class test/train/holdout split
def fit_score(data):
  global gbm0,drf0,glm0,dl0

  val = data[data["C1"] > 8] # Weeks 9
  train = data[data["C1"] <=8] # Weeks 3,4,5,6,7,8

  print("Training data has",train.ncol,"columns and",train.nrow,"rows, Validation data has",val.nrow,"rows")
  predictors_names = train.names
  predictors_names.remove("C6")
  
  # Run GBM
  s = time.time()
  
  gbm0 = H2OGradientBoostingEstimator(ntrees=500, # 500 works well
                                      max_depth=6,
                                      learn_rate=0.1)
    

  gbm0.train(x               =predictors_names,
             y               ="C6",
             training_frame  =train,
             validation_frame=val)

  gbm_elapsed = time.time() - s

  # Run DRF
  s = time.time()
    
  drf0 = H2ORandomForestEstimator(ntrees=250, max_depth=30)

  drf0.train(x               =predictors_names,
             y               ="C6",
             training_frame  =train,
             validation_frame=val)
    
  drf_elapsed = time.time() - s 
    
    
  # Run GLM
  s = time.time()

  glm0 = H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="poisson")
    
  glm0.train(x               =predictors_names,
             y               ="C6",
             training_frame  =train,
             validation_frame=val)

  glm_elapsed = time.time() - s
  
  # Run DL
  s = time.time()

  dl0 = H2ODeepLearningEstimator(hidden=[50,50,50,50], epochs=50)
    
  dl0.train(x               =predictors_names,
            y               ="C6",
            training_frame  =train,
            validation_frame=val)
    
  dl_elapsed = time.time() - s
  
  # ----------
  # 4- Score on holdout set & report
  train_r2_gbm = gbm0.model_performance(train).r2()
  val_r2_gbm  = gbm0.model_performance(val).r2()
  
  train_r2_drf = drf0.model_performance(train).r2()
  val_r2_drf  = drf0.model_performance(val).r2()
  
  train_r2_glm = glm0.model_performance(train).r2()
  val_r2_glm  = glm0.model_performance(val).r2()
    
  train_r2_dl = dl0.model_performance(train).r2()
  val_r2_dl  = dl0.model_performance(val).r2()
    
  # make a pretty HTML table printout of the results

  header = ["Model", "R2 TRAIN", "R2 TEST", "Model Training Time (s)"]
  table  = [
            ["GBM", train_r2_gbm, val_r2_gbm, round(gbm_elapsed,3)],
            ["DRF", train_r2_drf, val_r2_drf, round(drf_elapsed,3)],
            ["GLM", train_r2_glm, val_r2_glm, round(glm_elapsed,3)],
            ["DL ", train_r2_dl,  val_r2_dl,  round(dl_elapsed,3) ],
           ]
  h2o.display.H2ODisplay(table,header)
  # --------------

In [ ]:
# Split the data (into test & train), fit some models and predict on the holdout data
fit_score(train_hex)